In [370]:
import numpy as np
from cecfunction import F1, F3, F5, F6, F8
import time

In [371]:
POPULATION_SIZE = 15
MAX_ITERATION = 500
a = 2

In [372]:
functions = {
    "F1": [F1, -100, 100, 30],
    "F3": [F3, -100, 100, 30],
    "F5": [F5, -30, 30, 30],
    "F6": [F6, -100, 100, 30],
    "F8": [F8, -500, 500, 30]
}

In [373]:
obejctive_function = functions['F8']


In [374]:
def fitness_function(wolf: np.ndarray):
    return obejctive_function[0](wolf)

In [375]:
def update_position(omega: np.ndarray, alpha: np.ndarray, beta: np.ndarray, gamma: np.ndarray, a_update: float):

    A_alpha = 2 * a_update * np.random.random() - a_update
    A_beta = 2 * a_update * np.random.random() - a_update
    A_gamma = 2 * a_update * np.random.random() - a_update

    C_alpha = 2 * np.random.random()
    C_beta = 2 * np.random.random()
    C_gamma = 2 * np.random.random()

    D_alpha = np.abs(C_alpha * alpha - omega)
    D_beta = np.abs(C_beta * beta - omega)
    D_gamma = np.abs(C_gamma * gamma - omega)

    X_1 = alpha - A_alpha * D_alpha
    X_2 = beta - A_beta * D_beta
    X_3 = gamma - A_gamma * D_gamma

    return (X_1 + X_2 + X_3) / 3


In [376]:
results = []
st = time.time()
for _ in range(10):
    population = (np.random.random(size=(POPULATION_SIZE, obejctive_function[3])) * (obejctive_function[2] - obejctive_function[1])) + obejctive_function[1]
    for j in range(MAX_ITERATION):

        a = 2 * (1 - j ** 2 / MAX_ITERATION ** 2)
        disobeyment_size = int(np.round(POPULATION_SIZE * (1 - j ** 2 / MAX_ITERATION ** 2)))

        fitnesses = np.array([fitness_function(wolf) for wolf in population])
        sorted_args = fitnesses.argsort()
        leaders_idx = sorted_args[:3]
        alpha, beta, gamma = population[leaders_idx]
        
        population = population[sorted_args[3:]]
        if (POPULATION_SIZE - disobeyment_size > 4):
            obedient_population = np.array([update_position(wolf, alpha, beta, gamma, a) for wolf in population[:POPULATION_SIZE - disobeyment_size]])
            disobedient_population = np.array([update_position(wolf, alpha, beta, gamma, 2) for wolf in population[POPULATION_SIZE - disobeyment_size:]])

            if (len(disobedient_population) == 0):
                population = obedient_population
            else:
                population = np.concatenate([obedient_population, disobedient_population])

        else:
            population =  np.array([update_position(wolf, alpha, beta, gamma, 2) for wolf in population])
            
        population = np.concatenate([population, [alpha, beta, gamma]], axis= 0)

        population = population.clip(obejctive_function[1], obejctive_function[2])
    fitnesses = np.array([fitness_function(wolf) for wolf in population])
    results.append(np.min(fitnesses))
dur = time.time() - st

In [377]:
print('Mean: ', np.mean(results))
print('Std: ', np.std(results))
print('Duration: ', dur)


Mean:  -10064.649771531102
Std:  1926.7607566136508
Duration:  1.1300487518310547
